The Gibson Assembly® Protocol below is obtained from the New England Biolabs (NEB).
DOI: https://dx.doi.org/10.17504/protocols.io.bdddi226

In [38]:
import labop
from labop.execution_engine import ExecutionEngine
from labop_convert.opentrons.opentrons_specialization import OT2Specialization
from labop.utils.opentrons import run_ot2_sim, make_demo_script
from labop_convert.markdown.markdown_specialization import MarkdownSpecialization

import sbol3
import uml
import tyto
import json
import csv
import rdflib as rdfl
from IPython.display import Markdown, IFrame, Image

import os

# Set up our working directory for protocol files
out_dir = os.path.join(os.path.abspath(''), 'out')
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [39]:
# Setting up the document
# Set up the document
print('Setting up the document')
doc = sbol3.Document()
sbol3.set_namespace('http://bbn.com/synbio') # not sure what to put for namespace !!

# Import the primitive libraries
print('Importing the libraries')
labop.import_library('liquid_handling')
print('...Imported liquid handling')
labop.import_library('plate_handling')
print('...Imported plate handling')
labop.import_library('spectrophotometry')
print('...Imported spectrophotometry')
labop.import_library('sample_arrays')
print('...Imported sample arrays')

# Create the protocol
print('Creating protocol')
protocol = labop.Protocol('OT2_GibsonAssembly')
protocol.name = 'OT2 Gibson Assembly'
protocol.description = '''
This protocol is for Gibson Assembly of DNA fragments into plasmids using the New England Biolabs
Gibson Assembly® Protocol (E5510).
The protocol implements the specific case of 2-3 fragment assembly:
https://international.neb.com/protocols/2012/12/11/gibson-assembly-protocol-e5510
'''
doc.add(protocol)

Setting up the document
Importing the libraries
...Imported liquid handling
...Imported plate handling
...Imported spectrophotometry
...Imported sample arrays
Creating protocol


GIBSON ASSEMBLY PROCESS MAPPING
- P20 Single GEN2 / P20 Multi GEN2 (1-20uL) since total volume of reaction is 20uL and assembly reaction required in transformation is only 2uL
- 

In [40]:
# Create the materials to be provisioned

CONT_NS = rdfl.Namespace('https://sift.net/container-ontology/container-ontology#')
OM_NS = rdfl.Namespace('http://www.ontology-of-units-of-measure.org/resource/om-2/')

PREFIX_MAP = json.dumps({"cont": CONT_NS, "om": OM_NS})

d_h20 = sbol3.Component('deionized_H2O', '') # add identifier for deionized water
d_h20.name = 'Deionized Water'
doc.add(d_h20)

gib_mix = sbol3.Component('NEB_Gibson_AssemblyMasterMix', tyto.SBO.functional_entity)
gib_mix.name = 'NEB Gibson Assembly Master Mix'
gib_mix.derived_from.append('https://international.neb.com/products/e5510-gibson-assembly-cloning-kit#Product%20Information')
doc.add(gib_mix)

In [41]:
# Configure OT2 and load with labware
p20 = sbol3.Agent('p20_single_gen2', name='P20 Single GEN2')
doc.add(p20)

# not sure about the mounts !!
load = protocol.primitive_step('ConfigureRobot', instrument=OT2Specialization.EQUIPMENT['p20_single_gen2'], mount='left')
load = protocol.primitive_step('ConfigureRobot', instrument=OT2Specialization.EQUIPMENT['thermocycler'], mount='7')
tiprack_spec = labop.ContainerSpec('tiprack', 
                                   queryString='cont:Opentrons96TipRack300uL') # is this comaptible with P20 - 20uL tips?
load = protocol.primitive_step('LoadRackOnInstrument', rack=tiprack_spec, coordinates='1')
# only see 10uL, 300uL, 1000uL w/o filter and 10uL, 200uL, 1000uL w filter in container-ontology.ttl

# Specify layout of DNA source plate and build plate
dna_sources = protocol.input_value('source_samples', 'http://bioprotocols.org/labop#SampleCollection')
dna_build_layout = protocol.input_value('build_layout', 'http://bioprotocols.org/labop#SampleData')

# Set up plate for building
build_wells = protocol.primitive_step('DuplicateCollection', source=dna_build_layout)

# Set up plate with reaction
# reaction_plate = labop.ContainerSpec('reaction_plate', 
#                                      name='Reaction in 96-well plate',
#                                      queryString='cont:Corning96WellPlate360uLFlat',
#                                      prefixMap=PREFIX_MAP)
# load = protocol.primitive_step('LoadRackOnInstrument', rack=reaction_plate, coordinates='3')
# reaction_plate = protocol.primitive_step('EmptyContainer', specification=reaction_plate)


In [43]:
# Transfer DNA fragments into selected wells according to build plan
# protocol.primitive_step('TransferByMap', source=dna_sources, destination=build_wells.output_pin('samples'), plan=dna_build_layout)
transfer = protocol.primitive_step('Transfer', resource=dna_sources, destination=build_wells.output_pin('samples'),
                                   amount=sbol3.Measure(5, tyto.OM.microliter))
# unsure about the volume of DNA fragments cuz NEB says 0.2-0.5pmols * XuL
transfer.name = 'Add DNA fragments'

# Transfer deionized H20 into build wells
transfer = protocol.primitive_step('Transfer', resource=d_h20, destination=build_wells.output_pin('samples'),
                        amount=sbol3.Measure(10, tyto.OM.microliter))
transfer.name = 'Add deionized water'

# Transfer Gibson Assembly Master Mix into build wells
transfer = protocol.primitive_step('Transfer', resource=gib_mix, destination=build_wells.output_pin('samples'),
                        amount=sbol3.Measure(10, tyto.OM.microliter))
transfer.name = 'Add Gibson Assembly Master Mix'

# Seal and spin to mix
mix = protocol.primitive_step('Seal', location=build_wells.output_pin('samples'))
mix.name = 'Seal tubes'
mix = protocol.primitive_step('Spin', acceleration=sbol3.Measure(300, 'http://bioprotocols.org/temporary/unit/g'),
                        duration=sbol3.Measure(3, tyto.OM.minute))
mix.name = 'Spin to mix'
mix = protocol.primitive_step('Unseal', location=build_wells.output_pin('samples'))
mix.name = 'Unseal tubes'

ValueError: Could not find input pin named resource for Primitive Transfer

In [ ]:
# Incubation
incubate = protocol.primitive_step('Incubate', location=build_wells.output_pin('samples'),
                        duration=sbol3.Measure(15, tyto.OM.minute),
                        temperature=sbol3.Measure(50, tyto.OM.degree_Celsius))
incubate.name = 'Incubate'

In [ ]:
# Execute the protocol
filename = 'ot2_gibson_labop'
agent = sbol3.Agent('ot2_machine', name='OT2 machine')
ee = ExecutionEngine(specializations=[OT2Specialization(filename)], failsafe=False)
parameterValues = []
execution = ee.execute(protocol, agent, id='test_execution')

In [ ]:
# Print markdown on execution
print(execution.markdown)

In [ ]:
# Print operator instructions
Markdown(ee.specializations[0].markdown)

In [ ]:
# Print OT2 script
print(ee.specializations[0].script)